In [ ]:
!pip install numpy --upgrade --force-reinstall
!pip install torch --upgrade
!pip install transformers --upgrade
!pip install diffusers --upgrade
!pip install wikipedia requests librosa pathlib python-pptx TTS pillow google-generativeai gTTS
!pip install pandas openpyxl
import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 83.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.0 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.0 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 863.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import torch
import wikipedia
import requests
import os
import textwrap
import io
import pandas as pd
from pptx import Presentation
from pptx.util import Inches
from PIL import Image
from diffusers import StableDiffusionPipeline, DiffusionPipeline
from diffusers.utils import export_to_video
from TTS.api import TTS
import numpy as np
import google.generativeai as genai
from gtts import gTTS
from pathlib import Path

import hashlib
import datetime
import json

class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = str(timestamp)
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = json.dumps({
            "index": self.index,
            "timestamp": self.timestamp,
            "data": self.data,
            "previous_hash": self.previous_hash
        }, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        return Block(0, datetime.datetime.now(), "Genesis Block", "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, data):
        previous_block = self.get_latest_block()
        new_block = Block(len(self.chain), datetime.datetime.now(), data, previous_block.hash)
        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            curr = self.chain[i]
            prev = self.chain[i - 1]
            if curr.hash != curr.calculate_hash():
                return False
            if curr.previous_hash != prev.hash:
                return False
        return True

    def print_chain(self):
        for block in self.chain:
            print(f"Index: {block.index}, Hash: {block.hash}, Data: {block.data}, Time: {block.timestamp}")

    def export_to_excel(self, filename="blockchain_records.xlsx"):
      records = []
      for block in self.chain:
        data = block.data if isinstance(block.data, dict) else {"prompt": block.data}
        record = {
            "Index": block.index,
            "Hash": block.hash,
            "Previous Hash": block.previous_hash,
            "Prompt": data.get("prompt", ""),
            "Type": data.get("type", ""),
            "File": data.get("file", ""),
            "Timestamp": block.timestamp
        }
        records.append(record)
      new_df = pd.DataFrame(records)

      if os.path.exists(filename):
        # Read existing data
        existing_df = pd.read_excel(filename)
        # Merge and drop duplicates (optional: based on 'Index')
        merged_df = pd.concat([existing_df, new_df]).drop_duplicates(subset=["Index"], keep="last")
      else:
        merged_df = new_df

      merged_df.to_excel(filename, index=False)
      print(f"📒 Blockchain data appended and saved to '{filename}'")


print("Loading Gemini AI...")
genai.configure(api_key="Enter your api key")
gemini_model = genai.GenerativeModel(model_name="gemini-2.0-flash")

torch.backends.cuda.matmul.allow_tf32 = True

def generate_text(prompt):
    try:
        response = gemini_model.generate_content(prompt)
        # Some Gemini responses might not contain `.candidates[0].content.parts[0].text`
        # So we add checks and fallback logic
        if hasattr(response, "text"):
            text_response = response.text
        elif hasattr(response, "candidates"):
            parts = response.candidates[0].content.parts
            text_response = parts[0].text if parts else "No response text found."
        else:
            text_response = "No valid response received from Gemini."

        # Safely encode to avoid Unicode errors
        text_response = str(text_response).encode("utf-8", "ignore").decode("utf-8")

        print("📝 Generated text:\n", text_response)

        # Save MP3 using gTTS
        tts = gTTS(text_response)
        tts.save("generated_text.mp3")
        print("🔊 Response also saved as generated_text.mp3")

        return text_response

    except Exception as e:
        print(f"⚠️ Gemini Error: {str(e)}")
        return ""
    finally:
        torch.cuda.empty_cache()

def generate_image(prompt):
    try:
        print("🖼️ Generating image...")
        model_id = "stabilityai/stable-diffusion-2"
        stable_diffusion = StableDiffusionPipeline.from_pretrained(model_id).to("cuda")
        image = stable_diffusion(prompt, num_inference_steps=35, guidance_scale=9).images[0]
        image.save("generated_image.png")
        print("🖼️ Image generated and saved as generated_image.png")
    except Exception as e:
        print(f"Failed to generate image: {e}")
    finally:
        del stable_diffusion
        torch.cuda.empty_cache()

def generate_audio(text):
    try:
        print("🔈 Generating speech...")
        tts_model = TTS("tts_models/en/ljspeech/tacotron2-DDC").to("cuda")
        tts_model.tts_to_file(text=text, file_path="generated_sound.wav")
        print("🔈 Audio generated and saved as generated_sound.wav")
    except Exception as e:
        print(f"Failed to generate audio: {e}")
    finally:
        del tts_model
        torch.cuda.empty_cache()

def generate_video(prompt):
    try:
        print("🎬 Loading video generation pipeline...")
        video_pipe = DiffusionPipeline.from_pretrained(
            "cerspense/zeroscope_v2_576w",
            torch_dtype=torch.float16
        ).to("cuda")

        print("🎬 Generating video frames...")
        video_output = video_pipe(
            prompt=prompt,
            width=384,
            height=256,
            num_frames=24,
            num_inference_steps=25,
        ).frames

        frames = video_output[0]
        processed_frames = [
            np.clip(frame * 255, 0, 255).astype(np.uint8)
            for frame in frames
        ]

        export_to_video(processed_frames, "output.mp4", fps=24)
        print("🎥 Video saved as output.mp4")
    except Exception as e:
        print(f"Failed to generate video: {e}")
    finally:
        del video_pipe
        torch.cuda.empty_cache()

def get_best_wikipedia_page(topic):
    try:
        search_results = wikipedia.search(topic)
        return search_results[0] if search_results else topic
    except Exception as e:
        print(f"Error searching Wikipedia: {e}")
        return topic

def fetch_image(topic):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic.replace(' ', '_')}"
    try:
        response = requests.get(url).json()
        if "thumbnail" in response and "source" in response["thumbnail"]:
            return response["thumbnail"]["source"]
    except Exception as e:
        print(f"Error fetching image: {e}")
    return None

def download_image(image_url):
    #if not image_url:
        #return DEFAULT_IMAGE
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()
        img = Image.open(io.BytesIO(response.content))
        img_format = img.format.lower()
        if img_format not in ["jpeg", "png"]:
            print(f"Unsupported image format: {img_format}")
            #return DEFAULT_IMAGE
        img_path = f"topic_image.{img_format}"
        img.save(img_path)
        return img_path
    except Exception as e:
        print(f"Error downloading image: {e}")
        #return DEFAULT_IMAGE

def generate_slide_content(topic):
    topic = get_best_wikipedia_page(topic)
    try:
        summary = wikipedia.summary(topic, sentences=10)
        wrapped_text = textwrap.wrap(summary, width=200)
        return wrapped_text
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation Error: {e.options[:5]}")
    except wikipedia.exceptions.PageError:
        print("Page not found.")
    except Exception as e:
        print(f"Unexpected error: {e}")
    return None

def create_ppt(topic, content):
    prs = Presentation()
    title_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(title_slide_layout)
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    title.text = topic
    subtitle.text = "A Comprehensive Overview"

    image_url = fetch_image(topic)
    img_path = download_image(image_url)

    if img_path:
        slide.shapes.add_picture(img_path, Inches(6), Inches(1), Inches(3), Inches(2))

    slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    content_box = slide.placeholders[1]
    title.text = "Introduction"
    content_box.text = f"This presentation provides an overview of {topic}."

    subtopics = ["Background", "Main Concepts", "Applications", "Advantages", "Challenges", "Future Scope"]

    for i in range(len(subtopics)):
        slide_layout = prs.slide_layouts[1]
        slide = prs.slides.add_slide(slide_layout)
        title = slide.shapes.title
        content_box = slide.placeholders[1]
        title.text = subtopics[i]
        content_box.text = content[i] if i < len(content) else "More details available online."

    slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    content_box = slide.placeholders[1]
    title.text = "Conclusion"
    content_box.text = f"Thank you for exploring {topic}."

    ppt_filename = f"{topic.replace(' ', '_')}.pptx"
    prs.save(ppt_filename)
    print(f"📊 Presentation saved as {ppt_filename}")

    if img_path and img_path != DEFAULT_IMAGE:
        os.remove(img_path)
# Skipping their redefinition here for brevity

import ipywidgets as widgets
from IPython.display import display, Audio, Image as IPImage, Video as IPVideo, FileLink, clear_output

header = widgets.HTML(value="<h1 style='text-align:center;'>Generative AI with Blockchain</h1>")
display(header)

prompt_input = widgets.Textarea(
    value='',
    placeholder='Enter your prompt or topic here...',
    description='Prompt:',
    layout=widgets.Layout(width='100%', height='100px')
)
generate_type = widgets.Dropdown(
    options=['text', 'image', 'speech', 'video', 'ppt'],
    value='text',
    description='Generate:',
)
run_button = widgets.Button(description="Generate", button_style='success')
output_box = widgets.Output()
ai_chain = Blockchain()

def run_generation(_):
    with output_box:
        clear_output()
        prompt = prompt_input.value
        kind = generate_type.value
        data = {
            "prompt": prompt,
            "type": kind,
            "timestamp": str(datetime.datetime.now())
        }

        if kind == 'text':
          response = generate_text(prompt)
          data["file"] = "generated_text.txt"
          with open(data["file"], "w") as f:
            f.write(response)
            print("Generated Text:", response)

        elif kind == 'image':
            generate_image(prompt)
            data["file"] = "generated_image.png"
            display(IPImage("generated_image.png"))

        elif kind == 'speech':
            generate_audio(prompt)
            data["file"] = "generated_sound.wav"
            display(Audio("generated_sound.wav"))

        elif kind == 'video':
          generate_video(prompt)
          data["file"] = "output.mp4"
          display(IPVideo("output.mp4"))

        elif kind == 'ppt':
            content = generate_slide_content(prompt)
            if content:
                create_ppt(prompt, content)
                ppt_name = f"{prompt.replace(' ', '_')}.pptx"
                display(FileLink(ppt_name, result_html_prefix="Download: "))
            else:
                print("Could not generate PPT content.")


        ai_chain.add_block(data)
        print("\u2705 Action recorded on Blockchain.")
        ai_chain.export_to_excel()

run_button.on_click(run_generation)
view_chain_btn = widgets.Button(description="View Blockchain", button_style='info')


def display_chain(_):
    with output_box:
        clear_output(wait=True)
        ai_chain.print_chain()

view_chain_btn.on_click(display_chain)

ui = widgets.VBox([
    prompt_input,
    generate_type,
    run_button,
    view_chain_btn,
    output_box
])

display(ui)

Loading Gemini AI...


HTML(value="<h1 style='text-align:center;'>Generative AI with Blockchain</h1>")

In [ ]:
torch.cuda.empty_cache()
